## Accessing the Google analytics for CeNCOOS Data Data ##

This is configured to use a different key for Axiom owned service (which we have limited access to) and CeNCOOS owned services.

Additionally, each property has a different view, which are identified as the `VIEW_ID` value below. In this case `VIEW_ID` corresponds to CeNCOOS data services view.

`data-services: 180542384`<br>
`data-portal: 81137966`

In [2]:
import pandas as pd
import os
from googleapiclient.discovery import build
from google.oauth2 import service_account

In [3]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
BASE_DIR = '/home/pdaniel/web-analytics/'
KEY_FILE_LOCATION = os.path.join(BASE_DIR,'keys/resounding-axe-293817-00844a9aafb0.json')
VIEW_ID = "81137966"
FILE_NAME = 'portal-analytics-data.csv'

In [6]:
def initialize_analyticsreporting():
    """Establish a connection with the google reporting API V4"""
    credentials = service_account.Credentials.from_service_account_file(KEY_FILE_LOCATION)
    analytics = build('analyticsreporting', 'v4', credentials=credentials, cache_discovery=False)
    return analytics

def print_response(response):
    list = []
    # get report data
    for report in response.get('reports', []):
    # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])

    for row in rows:
        # create dict for each row
        dict = {}
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        # fill dict with dimension header (key) and dimension value (value)
        for header, dimension in zip(dimensionHeaders, dimensions):
            dict[header] = dimension

        # fill dict with metric header (key) and metric value (value)
        for i, values in enumerate(dateRangeValues):
            for metric, value in zip(metricHeaders, values.get('values')):
            #set int as int, float a float
                if ',' in value or '.' in value:
                    dict[metric.get('name')] = float(value)
                else:
                    dict[metric.get('name')] = int(value)

        list.append(dict)

    df = pd.DataFrame(list)
    return df


def get_report(analytics,metric):
    return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dimensions': [{'name': 'ga:date'}],
          'metrics': [{'expression': metric}],
          'dateRanges': [{'startDate': '30daysAgo', 'endDate': 'today'}]
        }]
      }
  ).execute()

def write_dataframe(df):
    out_df = df[['date','sessions','users']].copy(deep=True)
    out_df['date'] = pd.to_datetime(out_df['date'])
    out_df['sessions'] = out_df['sessions'].astype(str)
    out_df['users'] = out_df['users'].astype(str)
    return df
#     out_df.to_csv(os.path.join(BASE_DIR,FILE_NAME),index=False)

def copy_file_to_webserver():
    """Copy images from model runs to webserver where they can be viewed publically."""
    try:
        os.system('scp -i /etc/ssh/keys/pdaniel/scp_rsa {} skyrocket8.mbari.org:/var/www/html/data/web-analytics/ '.format(os.path.join(BASE_DIR,FILE_NAME)))
    except:
        logging.debug('Unabled to Copy Analytics File to Skyrocket')



In [10]:
def build_dataframe():
    metrics = [
        "ga:users",
        "ga:newUsers",
        "ga:percentNewSessions",
        "ga:sessions",
        "ga:bounces",
        "ga:sessionDuration",
        "ga:entrances",
        "ga:pageviews",
        "ga:pageviewsPerSession"
    ]
    for i, metric in enumerate(metrics):
        """ Build a dataframe for each varaible """
        response = get_report(initialize_analyticsreporting(),metric) # read data from a JSON format
        if i == 0:
            df = print_response(response)
        else:
            df[metric] = print_response(response)[metric]
            
    for col in df.columns:
        new_name = col.split(":")[-1]
        df.rename(columns={col:new_name},inplace=True)
    return write_dataframe(df)


In [11]:
df = build_dataframe()
# copy_file_to_webserver()

In [12]:
df

,date,users,newUsers,percentNewSessions,sessions,bounces,sessionDuration,entrances,pageviews,pageviewsPerSession
0,20201025,16,6,33.333333,18,6.0,5812.0,17,52,2.888889
1,20201026,30,12,33.333333,36,15.0,8024.0,36,90,2.500000
2,20201027,22,9,34.615385,26,8.0,4711.0,24,70,2.692308
3,20201028,32,14,41.176471,34,6.0,7627.0,34,69,2.029412
4,20201029,30,12,30.769231,39,9.0,11756.0,39,147,3.769231
5,20201030,20,10,34.482759,29,4.0,8498.0,27,83,2.862069
6,20201031,15,10,62.500000,16,2.0,4300.0,16,53,3.312500
7,20201101,8,3,27.272727,11,1.0,1901.0,10,17,1.545455
8,20201102,30,10,30.303030,33,4.0,7787.0,32,92,2.787879
9,20201103,24,8,26.666667,30,8.0,10681.0,30,93,3.100000
